<a href="https://colab.research.google.com/github/JasonThMdl/Thesis/blob/main/Preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pandas._libs.lib import map_infer_mask
import pandas as pd
import numpy as np
from google.colab import drive
from datetime import datetime,date,timedelta
import pickle



def get_date_range(df,span):
  min_date=df.min()
  max_date=df.max()
  list_dates=[]
  x=min_date

  while x<max_date:
    list_dates.append(x)
    x=x+timedelta(span)
  return list_dates


drive.mount('/content/drive')
path = "/content/drive/MyDrive/Thesis/New_Code/data.csv"
df = pd.read_csv(path)

df=df.drop(labels=['orders_id','customers_id'],axis=1) #drop useless columns
df=df[df['price_per_unit'] !=0 ]                       #drop rows where price equals 0
df=df.dropna()                                         #drop na rows
df=df.sort_values(by =['products_id','date_purchased'],kind='quicksort',na_position='first')
df.date_purchased=pd.to_datetime(df['date_purchased'],format='%Y-%m-%d %H:%M:%S')
df.date_purchased=pd.to_datetime(df['date_purchased']).dt.date
df=df.reset_index(drop=True)




list_dates=get_date_range(df.date_purchased,7)

print(len(df))
for i in range(len(df)):#len(df)
  x=1
  print('i',i)
  while True:
    if df.date_purchased[i]<list_dates[x] :
      df.date_purchased[i]=list_dates[x-1]
      break
    else:
      if x<len(list_dates)-1:
        x+=1
      else:
        df.date_purchased[i]=list_dates[len(list_dates)-1]
        break


df=df.groupby(['products_id','date_purchased']).agg({'products_quantity': np.sum, 'price_per_unit': np.mean})
df=df.reset_index()
price_mean=df.groupby(['products_id']).agg({'price_per_unit': np.mean})



In [ ]:
ss=0
dictionary_list = []
for i in (df.products_id.unique()):
  print('ss',ss)
  ss=ss+1
  for x in range(len(list_dates)):#len(list_dates)
    if not list_dates[x] in df[df['products_id']==i].date_purchased.values:
      new_row = {'products_id':i, 'date_purchased':list_dates[x], 'products_quantity':0, 'price_per_unit':0}
      dictionary_list.append(new_row)

df1 = pd.DataFrame.from_records(dictionary_list)
df= pd.concat([df1, df])

df=df.sort_values(by =['products_id','date_purchased'],kind='quicksort',na_position='first')
df=df.reset_index()
df=df.drop(labels=['index'],axis=1)

for i in range(len(df)):#len(df)
  if df.price_per_unit[i]==0:
   df.price_per_unit[i]=price_mean.loc[df.products_id[i]]


f=df.groupby(['products_id']).agg({'products_quantity': np.sum})
sorted=f.sort_values(['products_quantity'],ascending=False)
sorted=sorted.reset_index()

lista=[]

for i in range(len(sorted)):#len(sorted)
  print('i',i)
  lista.append(df[df['products_id']==sorted.products_id[i]])


df_final=pd.DataFrame()
df_final = pd.concat(lista)
df_final=df_final.reset_index(drop=True)

pickle_out = open("important","wb")
pickle.dump(df_final, pickle_out)
pickle_out.close()